In [7]:
# API - http://ec2-52-53-95-229.us-west-1.compute.amazonaws.com:8080/jointEOF.html
from io import BytesIO
import requests
import xarray as xr
import hvplot.xarray
from skimage import io
import cartopy.crs as ccrs 
import matplotlib.pyplot as plt

def download_data(url):
    r = requests.get(url)
    buf = BytesIO(r.content)
    return xr.open_dataset(buf,decode_times=False)

# Generate data remotely
cmda_url = 'http://ec2-52-53-95-229.us-west-1.compute.amazonaws.com:8080/svc/JointEOF'


query = dict(
    model1='GFDL_ESM2G',
    var1='clt',
    pres1=-999999,
    model2='GFDL_ESM2G',
    var2='hur',
    pres2=500,
    purpose='',
    lon1S=0,
    lon1E=360,
    lat1S=-90,
    lat1E=90,
    anomaly1=0,
    lon2S=0,
    lon2E=360,
    lat2S=-90,
    lat2E=90,
    timeS=200401,
    timeE=200412,
    anomaly2=0
)


r = requests.get(cmda_url, params=query)
print(r.url)
print(r.status_code)
# print(r.text)

http://ec2-52-53-95-229.us-west-1.compute.amazonaws.com:8080/svc/JointEOF?model1=GFDL_ESM2G&var1=clt&pres1=-999999&model2=GFDL_ESM2G&var2=hur&pres2=500&purpose=&lon1S=0&lon1E=360&lat1S=-90&lat1E=90&anomaly1=0&lon2S=0&lon2E=360&lat2S=-90&lat2E=90&timeS=200401&timeE=200412&anomaly2=0
200


In [4]:
# Download data into xarray Dataset object
data_url = r.json()['dataUrl']
ds = download_data(data_url)
print(ds.info())
ds

xarray.Dataset {
dimensions:
	lat1 = 90 ;
	lat2 = 90 ;
	lon1 = 144 ;
	lon2 = 144 ;
	mode = 3 ;
	time = 12 ;

variables:
	float32 time(time) ;
		time:units = months since 2004-01-01 00:00:00 ;
		time:calendar = gregorian ;
	float32 lon1(lon1) ;
		lon1:units = degrees_north ;
	float32 lat1(lat1) ;
		lat1:units = degrees_east ;
	float32 lon2(lon2) ;
		lon2:units = degrees_north ;
	float32 lat2(lat2) ;
		lat2:units = degrees_east ;
	float32 mode(mode) ;
	float32 pattern1(mode, lat1, lon1) ;
		pattern1:units = 1 ;
		pattern1:long_name = Total Cloud Fraction_pattern ;
	float32 pattern2(mode, lat2, lon2) ;
		pattern2:units = 1 ;
		pattern2:long_name = Relative Humidity_pattern ;
	float32 amp1(time, mode) ;
		amp1:units = % ;
		amp1:long_name = Total Cloud Fraction_amplitude ;
	float32 amp2(time, mode) ;
		amp2:units = % ;
		amp2:long_name = Relative Humidity_amplitude ;
	float32 covExplained(mode) ;

// global attributes:
	:source = Produced by CMDA analysis tools: JointEOF ;
}None


<xarray.Dataset>
Dimensions:       (lat1: 90, lat2: 90, lon1: 144, lon2: 144, mode: 3, time: 12)
Coordinates:
  * time          (time) float32 0.0 1.0 2.0 3.0 4.0 ... 7.0 8.0 9.0 10.0 11.0
  * lon1          (lon1) float32 1.25 3.75 6.25 8.75 ... 353.75 356.25 358.75
  * lat1          (lat1) float32 -89.494385 -87.97753 ... 87.97753 89.494385
  * lon2          (lon2) float32 1.25 3.75 6.25 8.75 ... 353.75 356.25 358.75
  * lat2          (lat2) float32 -89.494385 -87.97753 ... 87.97753 89.494385
  * mode          (mode) float32 0.0 1.0 2.0
Data variables:
    pattern1      (mode, lat1, lon1) float32 ...
    pattern2      (mode, lat2, lon2) float32 ...
    amp1          (time, mode) float32 ...
    amp2          (time, mode) float32 ...
    covExplained  (mode) float32 ...
Attributes:
    source:   Produced by CMDA analysis tools: JointEOF

In [9]:
ds.hvplot.quadmesh('lon1', 'lat1', widget_location='bottom', projection=ccrs.PlateCarree(), crs=ccrs.PlateCarree(), geo=True, coastline=True)

Column
    [0] HoloViews(DynamicMap, widget_location='bottom')
    [1] Row
        [0] HSpacer()
        [1] WidgetBox
            [0] DiscreteSlider(margin=(20, 20, 20, 20), name='mode', options=OrderedDict([('0', ...]), value=0.0, width=250)
        [2] HSpacer()

In [ ]:
ds.amp1.hvplot.line(x='time', y='amp1')

In [13]:
# data from api
image = io.imread(r.json()['url'])
f,ax = plt.subplots(1,figsize=(14,12))
ax.imshow(image)
plt.show()

URLError: <urlopen error [Errno 60] Operation timed out>